In [6]:
!pip install gym
!pip install gym-tetris 
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install gym[atari]
!pip install pyvirtualdisplay
!pip install piglet

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()



Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

### Dummy Observation PP

In [18]:
from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import SIMPLE_MOVEMENT,MOVEMENT

# Action space of SIMPLE_MOVEMENT
ACTION = {'N': 0, 'TL': 1, 'TR': 2, 'MR': 3, 'ML': 4, 'MD': 5}

class OPP:
  @staticmethod
  def getAction(state):
    # 1. Transform state into bitmap
    # 2. Pass to Conv network
    # 3. Return ACTION according to output of network
    return ACTION['MR']




class SPP:
  # State information
  environment = gym_tetris.make('TetrisA-v2')
  environment = JoypadSpace(environment, SIMPLE_MOVEMENT) # SIMPLE_MOVEMENT -> 6 actions (Nothing, TiltLeft, TiltRight, MoveLeft, MoveRight, MoveDown) 
  environment = wrap_env(environment)
  new_piece = True
  piece_info = {'T': 0, 'J': 0, 'Z': 0, 'O': 0, 'S': 0, 'L': 0, 'I': 0}
  steps = 0
  state = environment.reset()
  done = True
  
  

  @staticmethod
  def executeStep():
    if SPP.done:
      print("Game finished -> reset environment")
      SPP.new_piece = True
      SPP.piece_info = {'T': 0, 'J': 0, 'Z': 0, 'O': 0, 'S': 0, 'L': 0, 'I': 0}
      SPP.state = SPP.environment.reset()
    
    if SPP.new_piece:
      SPP.steps = 0
      SPP.new_piece = False

    # Get next action
    action = ACTION["MD"]
    if SPP.steps < 20:
      action = OPP.getAction(SPP.state)

    # Take action in environment and get next state
    SPP.state, reward, SPP.done, info = SPP.environment.step(action)
    SPP.steps += 1

    # Compare old piece_info with new piece_info
    shared_items = {k: SPP.piece_info[k] for k in SPP.piece_info if k in info["statistics"] and SPP.piece_info[k] == info["statistics"][k]}
    if len(shared_items) < 7:
      SPP.new_piece = True
      print(info["statistics"])

    SPP.piece_info = info["statistics"]

  @staticmethod
  def closeEnvironment():
    SPP.environment.close()
    show_video()

for i in range(500):
  SPP.executeStep()
SPP.closeEnvironment()

  



Game finished -> reset environment
{'T': 0, 'J': 0, 'Z': 0, 'O': 0, 'S': 1, 'L': 0, 'I': 0}
{'T': 0, 'J': 0, 'Z': 0, 'O': 0, 'S': 1, 'L': 0, 'I': 1}
{'T': 0, 'J': 0, 'Z': 1, 'O': 0, 'S': 1, 'L': 0, 'I': 1}
{'T': 0, 'J': 0, 'Z': 2, 'O': 0, 'S': 1, 'L': 0, 'I': 1}
{'T': 1, 'J': 0, 'Z': 2, 'O': 0, 'S': 1, 'L': 0, 'I': 1}
{'T': 1, 'J': 0, 'Z': 2, 'O': 0, 'S': 1, 'L': 0, 'I': 2}
{'T': 1, 'J': 0, 'Z': 2, 'O': 0, 'S': 2, 'L': 0, 'I': 2}
{'T': 1, 'J': 0, 'Z': 2, 'O': 1, 'S': 2, 'L': 0, 'I': 2}
{'T': 1, 'J': 0, 'Z': 2, 'O': 1, 'S': 2, 'L': 0, 'I': 3}
